In [1]:
%pip install sqlalchemy pymysql python-dotenv

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.8/2.1 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.3 MB/s  0:00:00

   -------------------- ------------------- 2/4 [greenlet]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ------------------------------ --------- 3/4 [sqlalchemy]
   ---------------------------------------- 4/4 [sqlalchemy]

Note: you may need to restart the 

In [5]:
import os
import pymysql

DB = {
    "host": os.getenv("MYSQL_HOST", "localhost"),
    "port": int(os.getenv("MYSQL_PORT", "3306")),
    "user": os.getenv("MYSQL_USER", "project_user"),
    "password": os.getenv("MYSQL_PASSWORD", "1234"),
    "database": os.getenv("MYSQL_DB", "project_db"),
    "charset": "utf8mb4",
    "local_infile": 1,
}

# 파일명(또는 키) -> (table, columns)
LOAD_PLAN = {
    "cw.csv": ("crosswalks", [
        "cw_uid","sido_code","district_code","address",
        "crosswalk_lat","crosswalk_lon","crosswalk_type","lane_count",
        "crosswalk_width","crosswalk_length","is_highland",
        "has_ped_signal","has_ped_button","has_ped_sound","has_bump","has_braille_block","has_spotlight"
    ]),
    "sg.csv": ("signals", [
        "sg_uid","sido_code","sigungu_code","address",
        "signal_lat","signal_lon","road_type","road_shape","is_main_road",
        "signal_type","has_ped_button","has_time_show","has_sound_signal"
    ]),
    "cw_sg_map.csv": ("crosswalk_signal_map", [
        "cw_uid","sg_uid","distance_m","confidence"
    ]),
    "acc.csv": ("accidents", [
        "acc_uid","sido_code","sigungu_code","year","month",
        "accident_count","casualty_count","fatality_count",
        "serious_injury_count","minor_injury_count","reported_injury_count"
    ]),
    "acc_hotspot.csv": ("accident_hotspots", [
        "accident_id","year","district_code","detail",
        "accident_count","casualty_count","fatality_count",
        "serious_injury_count","minor_injury_count","reported_injury_count",
        "accident_lon","accident_lat"
    ]),
    "district_all.csv": ("districts", [
        "district_code","district_name","available"
    ]),
    "cw_acc_map_500m.csv": ("cw_acc_map", [
        "cw_uid", "accident_id", "distance"
    ])
}

def load_one(cur, csv_path, table, columns, lines="\n", ignore=1):
    p = os.path.abspath(csv_path).replace("\\", "/")
    cols_sql = ", ".join(columns)
    sql = f"""
    LOAD DATA LOCAL INFILE '{p}'
    INTO TABLE {table}
    CHARACTER SET utf8mb4
    FIELDS TERMINATED BY ',' ENCLOSED BY '"'
    LINES TERMINATED BY '{lines}'
    IGNORE {ignore} LINES
    ({cols_sql});
    """
    cur.execute(sql)

def main(data_dir: str, lines_terminated_by: str = "\n", disable_fk_checks: bool = True):
    conn = pymysql.connect(**DB, autocommit=False)
    try:
        with conn.cursor() as cur:
            if disable_fk_checks:
                cur.execute("SET FOREIGN_KEY_CHECKS=0;")

            for filename, (table, columns) in LOAD_PLAN.items():
                path = os.path.join(data_dir, filename)
                if not os.path.exists(path):
                    print(f"[SKIP] not found: {path}")
                    continue

                load_one(cur, path, table, columns, lines=lines_terminated_by, ignore=1)
                print(f"[OK] {table} <= {filename}")

            if disable_fk_checks:
                cur.execute("SET FOREIGN_KEY_CHECKS=1;")

        conn.commit()
        print("ALL DONE")
    except Exception as e:
        conn.rollback()
        raise
    finally:
        conn.close()

if __name__ == "__main__":
    main(data_dir="C:/MiniProject/dataset", lines_terminated_by="\r\n")
    # Windows에서 CRLF면 lines_terminated_by="\r\n"


[OK] crosswalks <= cw.csv
[OK] signals <= sg.csv
[OK] crosswalk_signal_map <= cw_sg_map.csv
[OK] accidents <= acc.csv
[OK] accident_hotspots <= acc_hotspot.csv
[OK] districts <= district_all.csv
[OK] cw_acc_map <= cw_acc_map_500m.csv
ALL DONE
